In [1]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset('samsum')

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Train dataset size: 14732
Test dataset size: 819


In [2]:
from random import randrange        


sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"summary: \n{sample['summary']}\n---------------")

dialogue: 
Ethan: Which gas station has the best prices?
Alice: I always tank at the Tesco 
Sara: Which one?
Alice: The one out of town
Sara: I heard they have cheap gas there
Alice: It's always few cents per litre cheaper
Ethan: Good to know. Thanks!! 
---------------
summary: 
The Tesco that's out of town has the cheapest gas according to Alice and Sara.
---------------


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-small"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Max source length: 512
Max target length: 95


In [5]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [6]:
from transformers import AutoModelForSeq2SeqLM

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [7]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\linha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [9]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{'samsum'}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    fp16 = False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=10,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard"
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

  0%|          | 0/36830 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.9209, 'learning_rate': 4.993212055389628e-05, 'epoch': 0.01}
{'loss': 1.9404, 'learning_rate': 4.986424110779257e-05, 'epoch': 0.03}
{'loss': 1.9285, 'learning_rate': 4.9796361661688847e-05, 'epoch': 0.04}
{'loss': 1.7874, 'learning_rate': 4.972848221558512e-05, 'epoch': 0.05}
{'loss': 1.8714, 'learning_rate': 4.96606027694814e-05, 'epoch': 0.07}
{'loss': 1.8926, 'learning_rate': 4.9592723323377684e-05, 'epoch': 0.08}
{'loss': 1.7895, 'learning_rate': 4.952484387727396e-05, 'epoch': 0.1}
{'loss': 1.8691, 'learning_rate': 4.945696443117024e-05, 'epoch': 0.11}
{'loss': 1.8162, 'learning_rate': 4.938908498506653e-05, 'epoch': 0.12}
{'loss': 1.8048, 'learning_rate': 4.932120553896281e-05, 'epoch': 0.14}
{'loss': 1.8271, 'learning_rate': 4.9253326092859086e-05, 'epoch': 0.15}
{'loss': 1.8404, 'learning_rate': 4.9185446646755365e-05, 'epoch': 0.16}
{'loss': 1.8631, 'learning_rate': 4.9117567200651645e-05, 'epoch': 0.18}
{'loss': 1.8776, 'learning_rate': 4.9049687754547924e-05, 'ep

c:\Users\linha\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/205 [00:00<?, ?it/s]

...

  0%|          | 0/205 [00:00<?, ?it/s]

{'eval_loss': 1.617746114730835, 'eval_rouge1': 44.4606, 'eval_rouge2': 20.1139, 'eval_rougeL': 36.8539, 'eval_rougeLsum': 40.5396, 'eval_gen_len': 16.94871794871795, 'eval_runtime': 56.2279, 'eval_samples_per_second': 14.566, 'eval_steps_per_second': 3.646, 'epoch': 10.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


{'train_runtime': 6683.0116, 'train_samples_per_second': 22.044, 'train_steps_per_second': 5.511, 'train_loss': 1.5758347343757624, 'epoch': 10.0}


TrainOutput(global_step=36830, training_loss=1.5758347343757624, metrics={'train_runtime': 6683.0116, 'train_samples_per_second': 22.044, 'train_steps_per_second': 5.511, 'train_loss': 1.5758347343757624, 'epoch': 10.0})

In [11]:
#trainer.evaluate()

c:\Users\linha\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/205 [00:00<?, ?it/s]

{'eval_loss': 1.3694512844085693,
 'eval_rouge1': 47.5774,
 'eval_rouge2': 24.1721,
 'eval_rougeL': 40.1577,
 'eval_rougeLsum': 43.7501,
 'eval_gen_len': 17.024420024420024,
 'eval_runtime': 127.8578,
 'eval_samples_per_second': 6.406,
 'eval_steps_per_second': 1.603,
 'epoch': 10.0}